# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbcf083ad90>
├── 'a' --> tensor([[-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880]])
└── 'x' --> <FastTreeValue 0x7fbcf083ad30>
    └── 'c' --> tensor([[ 0.3599, -1.0203,  0.6055,  0.3434],
                        [ 0.7556, -0.7693, -1.4416, -0.9504],
                        [-0.7673, -0.3957,  0.3928,  0.9178]])

In [4]:
t.a

tensor([[-0.8240,  2.0740,  1.7117],
        [ 1.4386, -0.1540,  1.9880]])

In [5]:
%timeit t.a

63.2 ns ± 0.0487 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbcf083ad90>
├── 'a' --> tensor([[-0.0109, -0.0888, -0.2369],
│                   [ 1.0838, -2.0128, -0.8315]])
└── 'x' --> <FastTreeValue 0x7fbcf083ad30>
    └── 'c' --> tensor([[ 0.3599, -1.0203,  0.6055,  0.3434],
                        [ 0.7556, -0.7693, -1.4416, -0.9504],
                        [-0.7673, -0.3957,  0.3928,  0.9178]])

In [7]:
%timeit t.a = new_value

64 ns ± 0.0444 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8240,  2.0740,  1.7117],
               [ 1.4386, -0.1540,  1.9880]]),
    x: Batch(
           c: tensor([[ 0.3599, -1.0203,  0.6055,  0.3434],
                      [ 0.7556, -0.7693, -1.4416, -0.9504],
                      [-0.7673, -0.3957,  0.3928,  0.9178]]),
       ),
)

In [10]:
b.a

tensor([[-0.8240,  2.0740,  1.7117],
        [ 1.4386, -0.1540,  1.9880]])

In [11]:
%timeit b.a

53 ns ± 0.0439 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0570, -1.5393,  1.6000],
               [ 0.4585,  1.0593,  0.0380]]),
    x: Batch(
           c: tensor([[ 0.3599, -1.0203,  0.6055,  0.3434],
                      [ 0.7556, -0.7693, -1.4416, -0.9504],
                      [-0.7673, -0.3957,  0.3928,  0.9178]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.192 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 1.53 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 71 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 636 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 3.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbc454b4eb0>
├── 'a' --> tensor([[[-0.8240,  2.0740,  1.7117],
│                    [ 1.4386, -0.1540,  1.9880]],
│           
│                   [[-0.8240,  2.0740,  1.7117],
│                    [ 1.4386, -0.1540,  1.9880]],
│           
│                   [[-0.8240,  2.0740,  1.7117],
│                    [ 1.4386, -0.1540,  1.9880]],
│           
│                   [[-0.8240,  2.0740,  1.7117],
│                    [ 1.4386, -0.1540,  1.9880]],
│           
│                   [[-0.8240,  2.0740,  1.7117],
│                    [ 1.4386, -0.1540,  1.9880]],
│           
│                   [[-0.8240,  2.0740,  1.7117],
│                    [ 1.4386, -0.1540,  1.9880]],
│           
│                   [[-0.8240,  2.0740,  1.7117],
│                    [ 1.4386, -0.1540,  1.9880]],
│           
│                   [[-0.8240,  2.0740,  1.7117],
│                    [ 1.4386, -0.1540,  1.9880]]])
└── 'x' --> <FastTreeValue 0x7fbc454b4e80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbc454fff10>
├── 'a' --> tensor([[-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880],
│                   [-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880],
│                   [-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880],
│                   [-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880],
│                   [-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880],
│                   [-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880],
│                   [-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880],
│                   [-0.8240,  2.0740,  1.7117],
│                   [ 1.4386, -0.1540,  1.9880]])
└── 'x' --> <FastTreeValue 0x7fbc454ff370>
    └── 'c' --> tensor([[ 0.3599, -1.0203,  0.6055,  0.3434],
                        [ 0.7556, -0.7693, -1.4416, -0.9504],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 60.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.5 µs ± 72.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3599, -1.0203,  0.6055,  0.3434],
                       [ 0.7556, -0.7693, -1.4416, -0.9504],
                       [-0.7673, -0.3957,  0.3928,  0.9178]],
              
                      [[ 0.3599, -1.0203,  0.6055,  0.3434],
                       [ 0.7556, -0.7693, -1.4416, -0.9504],
                       [-0.7673, -0.3957,  0.3928,  0.9178]],
              
                      [[ 0.3599, -1.0203,  0.6055,  0.3434],
                       [ 0.7556, -0.7693, -1.4416, -0.9504],
                       [-0.7673, -0.3957,  0.3928,  0.9178]],
              
                      [[ 0.3599, -1.0203,  0.6055,  0.3434],
                       [ 0.7556, -0.7693, -1.4416, -0.9504],
                       [-0.7673, -0.3957,  0.3928,  0.9178]],
              
                      [[ 0.3599, -1.0203,  0.6055,  0.3434],
                       [ 0.7556, -0.7693, -1.4416, -0.9504],
                       [-0.7673, -0.3957,  0.3928,  0.9178]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3599, -1.0203,  0.6055,  0.3434],
                      [ 0.7556, -0.7693, -1.4416, -0.9504],
                      [-0.7673, -0.3957,  0.3928,  0.9178],
                      [ 0.3599, -1.0203,  0.6055,  0.3434],
                      [ 0.7556, -0.7693, -1.4416, -0.9504],
                      [-0.7673, -0.3957,  0.3928,  0.9178],
                      [ 0.3599, -1.0203,  0.6055,  0.3434],
                      [ 0.7556, -0.7693, -1.4416, -0.9504],
                      [-0.7673, -0.3957,  0.3928,  0.9178],
                      [ 0.3599, -1.0203,  0.6055,  0.3434],
                      [ 0.7556, -0.7693, -1.4416, -0.9504],
                      [-0.7673, -0.3957,  0.3928,  0.9178],
                      [ 0.3599, -1.0203,  0.6055,  0.3434],
                      [ 0.7556, -0.7693, -1.4416, -0.9504],
                      [-0.7673, -0.3957,  0.3928,  0.9178],
                      [ 0.3599, -1.0203,  0.6055,  0.3434],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 337 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
